# Structuring an API call 


In [ ]:
from openai import OpenAI
import os 
from IPython.display import Markdown

In [ ]:
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages =[{
        'role': 'user', 
        'content': 'Who developed chatgpt?'
    }]
)

print(response.choices[0].message.content)

# Challenges on a production environment

## Error Handling 
- Display useful error messages
- Alternatives for when the service is unavailable

## Moderation and Safety 
- Control unwanted inputs
- Minimizing the risk of data leaks

## Testing and validation 
- Checking for responses that are out of topic
- Testing for inconsisten behavior

## Communication with External Systems 
- Calling external functions and APIs
- Optimizing response times

# Specific Output Format


In [ ]:
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages =[{
        'role': 'user', 
        'content': 'Give me the best 3 novels of all time in json format'
    }]
)

Markdown(response.choices[0].message.content)

In [ ]:
# Create the request
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
   {"role": "user", "content": "Give me the 3 best graphic novels of all time in json format"}
  ],
  # Specify the response format
  response_format={"type": "json_object"}
)

# Print the response
print(response.choices[0].message.content)

# Handling errors


In [ ]:
response = client.chat.completions.create(
    model='text-davinci-001', 
    messages=[
        {
            "role": "user",
            "content": "give me the 3 best post rock albums of all time in json format"
        }
   ],
    response_format='json_object'
)

Markdown(response.choices[0].content.message)

## Connection Errors
Common expectable errors on either server or client side are **InternalServerError**, **APIConnectionError** or **APITimeourError**

In these cases: 
- Check connection configuration
- Reach out to support

## Resource Limits Errors 
**ConflictError** and **RateLimitError** 

- Check limit restrictions
- Ensure requests are within limits

## Authentication errors 
Wrong, expired or revoked api keys

## Bad Request errors
Check the expected request documentation for missing keys or typos.


# Handling Errors

In [ ]:
try: 
    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages= [{
            'role': 'user',
            'content': 'list five data science professions'
        }]
    )
except openai.AuthenticationError as e:
    print(f'OpenAI failed to authenticate: {e}')
    pass
except openai.RateLimitError as e:
    print(f'OpenAI API request exceeded rate limit: {e}')
    pass
except Exception as e: 
    print(f'Unable to generate a response. Exception: {e}')
    pass 



# RateErrorLimit 

The RateLimitError could be caused by: 
- Too many requests in a short period of time
- Too many tokens in the request

## Solutions 

### Retry (short wait between the requests)
Retry can be achieved with the python's tenacity library

In [ ]:
from tenacity import (retry, stop_after_attempt, wait_random_exponential)

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(2))
def get_response():
    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages= [{
            'role': 'user',
            'content': 'list five data science professions'
        }]
    )
    print(response.choices[0].message.content)

get_response()

### Batching (Processing multiple messages in one request)

In [ ]:
bands = ['Pearl Jam', 'Radiohead', 'Sigur Ros']

message = [
    {
        'role': 'system', 
        'content': '''You are given a series of rock bands and are asked to return 
        their best album along with the publication year. Provide each answer in the 
        response as a separate content''',
    }
]
[message.append({'role': 'user', 'content': band}) for band in bands]
print(message)

In [ ]:
response = client.chat.completions.create(
    model='gpt-4o-mini', 
    messages=message
)

In [ ]:
print(response.choices[0].message.content)

### Reducing the amount of tokens


In [ ]:
import tiktoken 

encoding = tiktoken.encoding_for_model('gpt-4o-mini')
prompt = 'tokens can be full words or groups of characters commonly grouped together: tokenization'

encoded = encoding.encode(prompt)
num_tokens = len(encoded)
print(f"Num tokens: {num_tokens}")
print(f"encoded: {encoded[:10]}")


In [ ]:
input_message = {"role": "user", "content": "I'd like to buy a shirt and a jacket. Can you suggest two color pairings for these items?"}

# Use tiktoken to create the encoding for your model
encoding = tiktoken.encoding_for_model('gpt-4o-mini')
# Check for the number of tokens
num_tokens = len(encoding.encode(input_message['content']))

# Run the chat completions function and print the response
if num_tokens <= 100:
    response = client.chat.completions.create(model="gpt-4o-mini", messages=[input_message])
    print(response.choices[0].message.content)
else:
    print("Message exceeds token limit")

# Defining Function Calling 

Functions help increasing the reliability

In [ ]:
function_definition = [
    {
        'type': 'function',
        'function': {
            'name': 'real_estate_info',
            'description': 'Get the information about homes for sale from the body of the input text',
            'parameters': {
                'type': 'object',
                'properties': {
                    'home type': {
                        'type': 'string',
                        'description': 'Home type'
                    },
                    'location': {
                        'type': 'string',
                        'description': 'Location'
                    },
                    'price': {
                        'type': 'integer',
                        'description': 'Price'
                    },
                    'bedrooms': {
                        'type': 'integer',
                        'description': 'Number of bedrooms'
                    }
                }
            }
        }
    }
]

message_listing =[
                    {'role': 'system',
                     'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
                    {'role': 'user',
                    'content': 'Step into this beautiful two-story, single-family home located in Springfield, USA, priced at $350,000. This charming property features 4 bedrooms, 2.5 bathrooms, a spacious living room with a cozy fireplace, a modern kitchen with stainless steel appliances, and a large backyard perfect for family gatherings. The master bedroom includes an en-suite bathroom and a walk-in closet. Enjoy the convenience of an attached two-car garage and a recently updated HVAC system. Located near top-rated schools, parks, and shopping centers, this home is ideal for families looking for comfort and convenience.'}
                ]

In [ ]:
response= client.chat.completions.create(
    model="gpt-4o-mini",
    # Add the message
    messages = message_listing,
    # Add your function definition
    tools=function_definition
)

In [ ]:
print(response.choices[0].message.tool_calls[0].function.arguments)

## Parallel function calling

In [ ]:
function_definition.append({
        'type': 'function',
        'function': {
            'name': 'real_estate_adj',
            'description': 'Get the main adjective used in the description',
            'parameters': {
                'type': 'object',
                'properties': {
                    'adjective': {
                        'type': 'string',
                        'description': 'Main adjective used in the description'
                    },
                }
            }
        }
    }
                          )

In [ ]:
response= client.chat.completions.create(
    model="gpt-4o-mini",
    # Add the message
    messages = message_listing,
    # Add your function definition
    tools=function_definition
)

In [ ]:
print(response.choices[0].message.tool_calls[0].function.arguments)

In [ ]:
print(response.choices[0].message.tool_calls[1].function.arguments)

If we end up having many function definition we could allow the LLM to pick the ones to use.

In [ ]:
response= client.chat.completions.create(
    model="gpt-4o-mini",
    messages = message_listing,
    tools=function_definition, 
    tool_choice='auto'
)

In [ ]:
print(response.choices[0].message.tool_calls[0].function.arguments)
print(response.choices[0].message.tool_calls[1].function.arguments)

If we want to be the ones choosing the function:


In [ ]:
response= client.chat.completions.create(
    model="gpt-4o-mini",
    messages = message_listing,
    tools=function_definition, 
    tool_choice={
        'type': 'function', 
                'function': {'name': 'real_estate_info'}
    }
)

In [ ]:
print(response.choices[0].message.tool_calls[0].function.arguments)
print(response.choices[0].message.tool_calls[1].function.arguments)

Sometimes the model makes assumptions about values that could end up being returned. To avoid this behaviour, we can use a system message asking the model not to do any assumptions.

When using this approach we have to be cautious since the response could contain empty dictionaries.

# Calling external APIS

In [ ]:
import requests 

def get_artwork(keyword): 
    url = 'https://api.artic.edu/api/v1/artworks/search'
    querystring = {'q': keyword} 
    response = requests.request('GET', url, params=querystring) 
    return response.text 


get_artwork('modern')

In [ ]:
function_definition = [
    {
        'type': 'function', 
        'function': {
            'name': 'get_artwork', 
            'description': ' This function calls the Art Institute of chicago api to find artwork that matches a keyword', 
            'parameters': {
                'type': 'object', 
                'properties': {
                    'artwork keyword': {
                        'type': 'string', 
                        'description': 'The keyword to be passed to the get_artwork function'
                    }
                }
            }, 
            'result': {'type': 'string'}
            
        }
    }
]

response = client.chat.completions.create(
    model = 'gpt-4o-mini', 
    messages = [
        {'role': 'system', 
        'content': 'You are an AI assistant, a specialist in history of art. You should interpret the user prompt, and based on it extract one keyword for recommending artwork related to their preference.'}, 
        {'role': 'user', 
        'content': 'I dont have much time to visit the museum and would like some recommendations. I like the seaside and quite places.'}    
    ], 
    
)


In [ ]:
print(response)

In [ ]:
import json 

if response.choices[0].finish_reason=='tool_calls': 
    function_call = response.choices[0].message.tool_calls[0].function 
    if function_call.name=='get_artwork': 
        artwork_keyword = json.loads(function_call.arguments)['artwork keyword']
        artwork = get_artwork(artwork_keyword)
        if artwork: 
            print(f"Here are some recommendations: {[i['title'] for i in json.loads(artwork)['data']]}")
        else: 
            print('Apoogies, I couldnt make any recommendations based on the request')
    
    else: 
        print('apologies, I couldnt find any artwork.')
else: 
    print('I am sorry but I could not understand your request.')

In [ ]:
response.choices[0].finish_reason

# Moderation


In [ ]:
message = "Can you show some example sentences in the past tense in French?"

# Use the moderation API
moderation_response = client.moderations.create(
    model = 'text-moderation-latest',
    input = message
)

# Print the response
print(moderation_response.results[0].categories)

In [ ]:
user_request = "Can you recommend a good restaurant in Berlin?"

# Write the system and user message
messages = [
    {'role': 'system',
    'content': 'You are a chatbot providing advice to tourists visiting Rome. Keep the topics limited to only covering questions about food and drink, attractions, history and things to do around the city. Check the user question and if it is allowed, provide a reply, otherwise provide the message: "Apologies, but I am not allowed to discuss this topic."'},
    {'role': 'user',
    'content': user_request}
]

response = client.chat.completions.create(
    model="gpt-4o-mini", messages=messages
)

# Print the response
print(response.choices[0].message.content)

# Validation

Adversarial testing is a valid way to identify flaws in the system before release.

# End user Ids 
OpenAI recommends to include unique user ids in the requests so any user violating rules could be identified.

In [ ]:
import uuid 

# Generate a unique ID
unique_id = str(uuid.uuid4())

response = client.chat.completions.create(  
  model="gpt-4o-mini", 
  messages=messages,
# Pass a user identification key
  user= unique_id
)

print(response.choices[0].message.content)